This script aims to add Stadtteil information

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

In [2]:
# Define input, output paths
current_dir = globals().get('current_dir', os.getcwd())
input_path = os.path.join(current_dir, '../data/mobidata/raw/csv_files/')
map_path = os.path.join(current_dir, '../data/map/Stadtteile_SHP/Stadtviertel_Karlsruhe.shp')
output_path = os.path.join(current_dir, '../data/mobidata/processed/')

# Define file path
input_file = 'baustellen_vorschau.csv'
input_file_path = os.path.join(input_path, input_file)

In [12]:
# 1) Load data 
df = pd.read_csv(input_file_path)


In [14]:
df

,type,totalFeatures,numberMatched,numberReturned,timeStamp,crs_type,crs_properties_name,feature_type,feature_id,geometry_name,...,art,lage,tagesbaustelle,verursacher,zusatzinfo,sperrung,projektnummer,vorgangsnummer,datenquelle,stand
0,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1112,geom,...,AK5_encours,D105 Hésingue/St-Louis,NaN,NaN,Dévoiement de la chaussée et changements des l...,NaN,6769,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
1,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1117,geom,...,CE3a Information,"D130 - Bœrsch, Grendelbruch, Le Hohwald, Natzw...",NaN,NaN,Manifestation sportive intitulée “Biathlon <br...,NaN,6795,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
2,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1141,geom,...,KC1_route-barree,D426 - Entre Obernai et Ottrott,NaN,NaN,Route barrée avec mise en place d'une déviatio...,NaN,6810,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
3,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1142,geom,...,Travaux,D426 - Gerstheim,NaN,NaN,Veuillez vous référer aux documents joints pou...,NaN,6805,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
4,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1148,geom,...,KC1_route-barree,D505 Soultz-Haut-Rhin,NaN,NaN,Travaux de voirie;<br />Veuillez consulter les...,NaN,6813,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314525,geom,...,Kraneinsatz,Fettweisstraße zw. Industriestraße und Hohleic...,NaN,Privat,NaN,mit Vollsperrung,53234,2025V5726,Stadt Karlsruhe,2025-09-30T22:00:00Z
266,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314547,geom,...,Fernwärmeversorgung,Augartenstraße zw. Wilhemstraße und Marienstraße,NaN,Stadtwerke Karlsruhe,NaN,mit Vollsperrung,45072,2025V5729,Stadt Karlsruhe,2025-09-30T22:00:00Z
267,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314548,geom,...,Fernwärmeversorgung,Augartenstraße zw. Wilhemstraße und Marienstraße,NaN,Stadtwerke Karlsruhe,NaN,mit Vollsperrung,45072,2025V5729,Stadt Karlsruhe,2025-09-30T22:00:00Z
268,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314747,geom,...,Fernwärmeversorgung,Bulacher Straße zw. Breite Straße und Bulacher...,NaN,Stadtwerke Karlsruhe,Engstelle 1-streifige Verkehrsführung im Baube...,mit Verkehrsbehinderung,52845,2025V5310,Stadt Karlsruhe,2025-10-05T22:00:00Z


In [18]:
# Method 3:
from shapely.geometry import Point, shape

# Make sure x/y are numeric (strings -> numbers; bad values -> NaN)
for c in ["geometry_x", "geometry_y"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

def _parse_coords(val):
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return None
    if isinstance(val, (list, tuple)):
        return val
    if isinstance(val, str):
        s = val.strip()
        if not s or s.lower() == "nan":
            return None
        # try JSON first, then Python literal
        try:
            return json.loads(s)
        except Exception:
            return ast.literal_eval(s)
    return None

def _make_geom(row):
    gtype = str(row.get("geometry_type", "")).strip()
    # Points: prefer numeric x/y if available
    if gtype == "Point" and pd.notna(row.get("geometry_x")) and pd.notna(row.get("geometry_y")):
        return Point(float(row["geometry_x"]), float(row["geometry_y"]))
    # Fallback / non-points: parse coordinates string
    coords = _parse_coords(row.get("geometry_coordinates"))
    if coords is None or not gtype:
        return None
    return shape({"type": gtype, "coordinates": coords})

df["geometry"] = df.apply(_make_geom, axis=1)

# Build GeoDataFrame (keep only rows that produced a geometry)
gdf_3 = gpd.GeoDataFrame(df[df["geometry"].notna()].copy(), geometry="geometry", crs="EPSG:25832")

In [19]:
gdf_3

,type,totalFeatures,numberMatched,numberReturned,timeStamp,crs_type,crs_properties_name,feature_type,feature_id,geometry_name,...,lage,tagesbaustelle,verursacher,zusatzinfo,sperrung,projektnummer,vorgangsnummer,datenquelle,stand,geometry
0,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1112,geom,...,D105 Hésingue/St-Louis,NaN,NaN,Dévoiement de la chaussée et changements des l...,NaN,6769,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (389426.383 5270752.731)
1,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1117,geom,...,"D130 - Bœrsch, Grendelbruch, Le Hohwald, Natzw...",NaN,NaN,Manifestation sportive intitulée “Biathlon <br...,NaN,6795,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (373668.451 5366749.022)
2,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1141,geom,...,D426 - Entre Obernai et Ottrott,NaN,NaN,Route barrée avec mise en place d'une déviatio...,NaN,6810,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (385210.785 5368673.758)
3,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1142,geom,...,D426 - Gerstheim,NaN,NaN,Veuillez vous référer aux documents joints pou...,NaN,6805,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (405143.429 5359769.028)
4,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1148,geom,...,D505 Soultz-Haut-Rhin,NaN,NaN,Travaux de voirie;<br />Veuillez consulter les...,NaN,6813,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (366913.581 5304229.635)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314525,geom,...,Fettweisstraße zw. Industriestraße und Hohleic...,NaN,Privat,NaN,mit Vollsperrung,53234,2025V5726,Stadt Karlsruhe,2025-09-30T22:00:00Z,POINT (450461.305 5428702.401)
266,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314547,geom,...,Augartenstraße zw. Wilhemstraße und Marienstraße,NaN,Stadtwerke Karlsruhe,NaN,mit Vollsperrung,45072,2025V5729,Stadt Karlsruhe,2025-09-30T22:00:00Z,"POLYGON ((456710.761 5427527.977, 456705.761 5..."
267,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314548,geom,...,Augartenstraße zw. Wilhemstraße und Marienstraße,NaN,Stadtwerke Karlsruhe,NaN,mit Vollsperrung,45072,2025V5729,Stadt Karlsruhe,2025-09-30T22:00:00Z,POINT (456711.478 5427524.471)
268,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314747,geom,...,Bulacher Straße zw. Breite Straße und Bulacher...,NaN,Stadtwerke Karlsruhe,Engstelle 1-streifige Verkehrsführung im Baube...,mit Verkehrsbehinderung,52845,2025V5310,Stadt Karlsruhe,2025-10-05T22:00:00Z,"POLYGON ((455051.384 5426794.19, 455049.206 54..."


In [20]:
df_list = df.to_dict('records')
df_list

[{'type': 'FeatureCollection',
  'totalFeatures': 270,
  'numberMatched': 270,
  'numberReturned': 270,
  'timeStamp': '2025-10-06T14:15:33.080Z',
  'crs_type': 'name',
  'crs_properties_name': 'urn:ogc:def:crs:EPSG::25832',
  'feature_type': 'Feature',
  'feature_id': 'baustellen_vorschau.1112',
  'geometry_name': 'geom',
  'geometry_type': 'Point',
  'geometry_coordinates': '[389426.38255268, 5270752.73090236]',
  'geometry_x': 389426.38255268,
  'geometry_y': 5270752.73090236,
  'id': 1112,
  'gemeinde': nan,
  'vorgangszeitraum_von': '2025-10-09T22:00:00Z',
  'vorgangszeitraum_bis': '2025-10-12T18:00:00Z',
  'art': 'AK5_encours',
  'lage': 'D105 Hésingue/St-Louis',
  'tagesbaustelle': nan,
  'verursacher': nan,
  'zusatzinfo': "Dévoiement de la chaussée et changements des limitations de vitesses - voir mesures d'exploitations ci-joints.",
  'sperrung': nan,
  'projektnummer': 6769,
  'vorgangsnummer': nan,
  'datenquelle': 'Collectivité européenne d’Alsace',
  'stand': '2025-10-05T

In [21]:
# 2) Load Stadtteile polygons
gdf_stadtteile = gpd.read_file(map_path)

In [22]:
gdf_stadtteile

,OBJECTID,NUMMER,NAME,BEVD,SHAPE_AREA,SHAPE_LEN,geometry
0,47,132,Bulach,0.0,2.349710e+06,8835.931194,"POLYGON ((454615.549 5426795.403, 454620.722 5..."
1,48,122,Waldlage,0.0,6.152226e+05,3868.385783,"POLYGON ((453802.311 5426791.226, 453799.052 5..."
2,49,115,Neue Heidenstückersiedlung,0.0,7.117038e+05,4693.388222,"POLYGON ((453219.232 5427203.653, 453219.566 5..."
3,50,112,Hardecksiedlung,0.0,4.739133e+05,3617.545619,"POLYGON ((453910.998 5427616.596, 453907.044 5..."
4,51,111,Alt-Grünwinkel,0.0,1.141327e+06,4960.769418,"POLYGON ((453643.475 5428138.041, 453640.67 54..."
...,...,...,...,...,...,...,...
65,66,071,Nördlicher Teil,0.0,1.377227e+06,5216.090139,"POLYGON ((458149.345 5430144.52, 458151.172 54..."
66,67,195,Aue,0.0,2.187323e+06,7553.954377,"POLYGON ((460626.689 5425626.578, 460624.636 5..."
67,68,191,Alt-Durlach,0.0,5.583243e+06,13576.175961,"POLYGON ((462708.966 5428419.393, 462706.385 5..."
68,69,161,Waldlage,0.0,9.477397e+06,14302.383936,"POLYGON ((460151.186 5432841.279, 460150.673 5..."


In [23]:

# Keep only the fields we need and give clear names
gdf_stadtteile = gdf_stadtteile.rename(columns={
    "NAME": "Stadtteil",
    "NUMMER": "Stadtteil_Nummer"
})

In [24]:
gdf_stadtteile

,OBJECTID,Stadtteil_Nummer,Stadtteil,BEVD,SHAPE_AREA,SHAPE_LEN,geometry
0,47,132,Bulach,0.0,2.349710e+06,8835.931194,"POLYGON ((454615.549 5426795.403, 454620.722 5..."
1,48,122,Waldlage,0.0,6.152226e+05,3868.385783,"POLYGON ((453802.311 5426791.226, 453799.052 5..."
2,49,115,Neue Heidenstückersiedlung,0.0,7.117038e+05,4693.388222,"POLYGON ((453219.232 5427203.653, 453219.566 5..."
3,50,112,Hardecksiedlung,0.0,4.739133e+05,3617.545619,"POLYGON ((453910.998 5427616.596, 453907.044 5..."
4,51,111,Alt-Grünwinkel,0.0,1.141327e+06,4960.769418,"POLYGON ((453643.475 5428138.041, 453640.67 54..."
...,...,...,...,...,...,...,...
65,66,071,Nördlicher Teil,0.0,1.377227e+06,5216.090139,"POLYGON ((458149.345 5430144.52, 458151.172 54..."
66,67,195,Aue,0.0,2.187323e+06,7553.954377,"POLYGON ((460626.689 5425626.578, 460624.636 5..."
67,68,191,Alt-Durlach,0.0,5.583243e+06,13576.175961,"POLYGON ((462708.966 5428419.393, 462706.385 5..."
68,69,161,Waldlage,0.0,9.477397e+06,14302.383936,"POLYGON ((460151.186 5432841.279, 460150.673 5..."


In [25]:
gdf_stadtteile.crs

<Projected CRS: EPSG:25832>
Name: ETRS89 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 6°E and 12°E: Austria; Denmark - onshore and offshore; Germany - onshore and offshore; Italy - onshore and offshore; Norway including Svalbard - onshore and offshore; Spain - offshore.
- bounds: (6.0, 36.53, 12.01, 84.01)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [26]:
# 3) Ensure both layers share the same CRS
if gdf_stadtteile.crs is None:
    # Only set if you are certain the polygons are in EPSG:25832
    gdf_stadtteile = gdf_stadtteile.set_crs("EPSG:25832")
elif gdf_stadtteile.crs.to_epsg() != 25832:
    gdf_stadtteile = gdf_stadtteile.to_crs(25832)

In [27]:
gdf_stadtteile

,OBJECTID,Stadtteil_Nummer,Stadtteil,BEVD,SHAPE_AREA,SHAPE_LEN,geometry
0,47,132,Bulach,0.0,2.349710e+06,8835.931194,"POLYGON ((454615.549 5426795.403, 454620.722 5..."
1,48,122,Waldlage,0.0,6.152226e+05,3868.385783,"POLYGON ((453802.311 5426791.226, 453799.052 5..."
2,49,115,Neue Heidenstückersiedlung,0.0,7.117038e+05,4693.388222,"POLYGON ((453219.232 5427203.653, 453219.566 5..."
3,50,112,Hardecksiedlung,0.0,4.739133e+05,3617.545619,"POLYGON ((453910.998 5427616.596, 453907.044 5..."
4,51,111,Alt-Grünwinkel,0.0,1.141327e+06,4960.769418,"POLYGON ((453643.475 5428138.041, 453640.67 54..."
...,...,...,...,...,...,...,...
65,66,071,Nördlicher Teil,0.0,1.377227e+06,5216.090139,"POLYGON ((458149.345 5430144.52, 458151.172 54..."
66,67,195,Aue,0.0,2.187323e+06,7553.954377,"POLYGON ((460626.689 5425626.578, 460624.636 5..."
67,68,191,Alt-Durlach,0.0,5.583243e+06,13576.175961,"POLYGON ((462708.966 5428419.393, 462706.385 5..."
68,69,161,Waldlage,0.0,9.477397e+06,14302.383936,"POLYGON ((460151.186 5432841.279, 460150.673 5..."


In [29]:
# 4) Spatial join (points within polygons)
# Keep only the column we need from gdf_stadtteile
gdf_stadtteile_subset = gdf_stadtteile[["Stadtteil_Nummer","Stadtteil", "geometry"]]

# Spatial join
gdf_joined = gpd.sjoin(gdf_3, gdf_stadtteile_subset, how="left", predicate="within")

In [30]:
# Check which column is used as geometry
print(gdf_3.geometry.name)       # typically 'geometry'
print(gdf_stadtteile_subset.geometry.name)

geometry
geometry


✋ The cell elow need to be adjusted, now the geotype is not just "Points" but also eg. "Polygon"... 

In [31]:
# 5) If some points lie exactly on borders and return NaN, try a tiny buffer:
if gdf_joined["Stadtteil"].isna().any():
    # buffer(0) cleans invalid geometries; buffer(0.1) grows by 0.1m in EPSG:25832 units
    gdf_points_buf = gdf_points.copy()
    gdf_points_buf["geometry"] = gdf_points_buf.buffer(0.1)
    gdf_joined_fallback = gpd.sjoin(gdf_points_buf, gdf_stadtteile, how="left", predicate="intersects")
    # fill only the missing ones
    for col in ["Stadtteil", "Stadtteil_Nummer"]:
        gdf_joined[col] = gdf_joined[col].fillna(gdf_joined_fallback[col])



In [32]:
# 6) Display the joined variable
gdf_joined

,type,totalFeatures,numberMatched,numberReturned,timeStamp,crs_type,crs_properties_name,feature_type,feature_id,geometry_name,...,zusatzinfo,sperrung,projektnummer,vorgangsnummer,datenquelle,stand,geometry,index_right,Stadtteil_Nummer,Stadtteil
0,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1112,geom,...,Dévoiement de la chaussée et changements des l...,NaN,6769,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (389426.383 5270752.731),NaN,NaN,NaN
1,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1117,geom,...,Manifestation sportive intitulée “Biathlon <br...,NaN,6795,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (373668.451 5366749.022),NaN,NaN,NaN
2,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1141,geom,...,Route barrée avec mise en place d'une déviatio...,NaN,6810,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (385210.785 5368673.758),NaN,NaN,NaN
3,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1142,geom,...,Veuillez vous référer aux documents joints pou...,NaN,6805,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (405143.429 5359769.028),NaN,NaN,NaN
4,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.1148,geom,...,Travaux de voirie;<br />Veuillez consulter les...,NaN,6813,NaN,Collectivité européenne d’Alsace,2025-10-05T22:00:00Z,POINT (366913.581 5304229.635),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314525,geom,...,NaN,mit Vollsperrung,53234,2025V5726,Stadt Karlsruhe,2025-09-30T22:00:00Z,POINT (450461.305 5428702.401),49.0,083,Rheinhafen
266,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314547,geom,...,NaN,mit Vollsperrung,45072,2025V5729,Stadt Karlsruhe,2025-09-30T22:00:00Z,"POLYGON ((456710.761 5427527.977, 456705.761 5...",16.0,032,Südlicher Teil
267,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314548,geom,...,NaN,mit Vollsperrung,45072,2025V5729,Stadt Karlsruhe,2025-09-30T22:00:00Z,POINT (456711.478 5427524.471),16.0,032,Südlicher Teil
268,FeatureCollection,270,270,270,2025-10-06T14:15:33.080Z,name,urn:ogc:def:crs:EPSG::25832,Feature,baustellen_vorschau.102314747,geom,...,Engstelle 1-streifige Verkehrsführung im Baube...,mit Verkehrsbehinderung,52845,2025V5310,Stadt Karlsruhe,2025-10-05T22:00:00Z,"POLYGON ((455051.384 5426794.19, 455049.206 54...",30.0,131,Beiertheim


In [34]:
# save the result as a csv file
output_file = 'baustellen_vorschau.csv'
output_file_path = os.path.join(output_path, output_file)
gdf_joined.to_csv(output_file_path, index=False)